In [26]:
import pandas as pd
import datetime
import os
from pathlib import Path 

In [25]:
files = os.listdir("data/source")
# loop through files to concat into one dataset
i = 1
for file in files:
    path = os.path.join("data/source", file)
    df = pd.read_csv(path)

    # convert times to datetime to do a timedelta to get trip length
    df['started_at']= pd.to_datetime(df['started_at'])
    df['ended_at']= pd.to_datetime(df['ended_at'])
    df['trip_length'] = (df['ended_at'] - df['started_at'])

    # convert trip length to minutes since tableau doesn't understand timedelta
    df['trip_length_min'] = (df['ended_at'] - df['started_at'])/ datetime.timedelta(minutes=1)

    # break out start/end day of week
    df['start_dow'] = df['started_at'].dt.weekday
    df['end_dow'] = df['ended_at'].dt.weekday

    #break out month
    df['month_num'] = df['started_at'].dt.month

    # keep only trips at least 1 minute long
    df = df.loc[df['trip_length_min'] >= 1]

    # create dataframe
    if i == 1:
        trips_2022 = df
        print(path)
    else:
        trips_2022 = pd.concat([trips_2022, df])
        print(path)
    i += 1
        

data/source/JC-202203-citibike-tripdata.csv
data/source/JC-202201-citibike-tripdata.csv
data/source/JC-202209-citibike-tripdata.csv
data/source/JC-202207-citbike-tripdata.csv
data/source/JC-202205-citibike-tripdata.csv
data/source/JC-202202-citibike-tripdata.csv
data/source/JC-202208-citibike-tripdata.csv
data/source/JC-202206-citibike-tripdata.csv
data/source/JC-202204-citibike-tripdata.csv


In [27]:
trips_2022.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_length,trip_length_min,start_dow,end_dow,month_num
0,3255D3E3F33CDC45,classic_bike,2022-03-18 15:38:17,2022-03-18 15:45:34,Mama Johnson Field - 4 St & Jackson St,HB404,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,40.743140,-74.040041,40.736982,-74.027781,casual,0 days 00:07:17,7.283333,4,4,3
1,17FA5604A37338F9,electric_bike,2022-03-04 16:44:48,2022-03-04 16:50:45,Baldwin at Montgomery,JC020,Grove St PATH,JC005,40.723659,-74.064194,40.719586,-74.043117,member,0 days 00:05:57,5.950000,4,4,3
2,7DEC9ADDB8D6BBE1,electric_bike,2022-03-13 17:44:32,2022-03-13 17:54:44,Baldwin at Montgomery,JC020,Grove St PATH,JC005,40.723659,-74.064194,40.719586,-74.043117,member,0 days 00:10:12,10.200000,6,6,3
3,9D69F74EEF231A2E,classic_bike,2022-03-13 15:33:47,2022-03-13 15:41:22,Baldwin at Montgomery,JC020,Grove St PATH,JC005,40.723659,-74.064194,40.719586,-74.043117,member,0 days 00:07:35,7.583333,6,6,3
4,C84AE4A9D78A6347,classic_bike,2022-03-11 12:21:18,2022-03-11 12:33:24,Baldwin at Montgomery,JC020,Grove St PATH,JC005,40.723659,-74.064194,40.719586,-74.043117,member,0 days 00:12:06,12.100000,4,4,3


In [29]:
filepath = Path('data/trips_2022.csv')   
trips_2022.to_csv(filepath, index=False)

In [ ]:
start_station = trips_2022[['start_station_name','start_lat','start_lng']].copy()
start_station_gb = start_station.groupby(['start_station_name'],as_index=False).size()
start_station_gb = start_station_gb.rename(columns={"size":"start_count"})
start_station = start_station.drop_duplicates(subset=['start_station_name'])
start_station.head()

In [ ]:
start_station_gb.head()

In [ ]:
start_station = pd.merge(start_station_gb, start_station, on="start_station_name")

In [ ]:
end_station = trips_2022[['end_station_name','end_lat','end_lng']].copy()
end_station_gb = end_station.groupby(['end_station_name'],as_index=False).size()
end_station_gb = end_station_gb.rename(columns={"size":"end_count"})
end_station = end_station.drop_duplicates(subset=['end_station_name'])
end_station.head()

In [ ]:
end_station = pd.merge(end_station_gb, end_station, on="end_station_name")

In [ ]:
station_counts = pd.merge(start_station, end_station, how='outer', left_on=["start_station_name"], right_on=["end_station_name"])
station_counts[station_counts.isna().any(axis=1)]
station_counts

In [ ]:
station_counts = station_counts[['end_station_name','start_count','end_count', 'end_lat','end_lng']].copy().fillna(0)
station_counts = station_counts.rename(columns={'end_station_name':'station_name','end_lat':'lat', 'end_lng':'lng'})
station_counts

In [ ]:
filepath = Path('data/stations_2022.csv')   
station_counts.to_csv(filepath, index=False)

In [ ]:
station_counts.loc[station_counts['start_count'] == 0]

In [ ]:
trips_2022.loc[trips_2022['rideable_type'] == 'docked_bike']